In [16]:
import vectorbt as vbt
import yfinance
import datetime
import pandas as pd
import numpy as np

In [17]:
end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=2)

In [18]:
btc_price = vbt.YFData.download(
    'ETH-USD',
    missing_index = 'drop',
    start = start_time,
    end = end_time,
    interval= "1m").get('Close')

In [20]:
btc_price.head()

Datetime
2023-09-30 05:16:00+00:00    1671.835327
2023-09-30 05:17:00+00:00    1672.096680
2023-09-30 05:18:00+00:00    1672.276123
2023-09-30 05:19:00+00:00    1672.083618
2023-09-30 05:20:00+00:00    1672.021362
Name: Close, dtype: float64

In [21]:
def custom_indicator(close,window=14):
    rsi = vbt.RSI.run(close,window=window)
    return rsi.rsi

In [23]:
ind = vbt.IndicatorFactory(
    class_name = "Combination",
    short_name = 'comb',
    input_names= ['close'],
    param_names=  ['window'],
    output_names = ['value']
    ).wfrom_apply_func(
                custom_indicator,
                window = 14
                )

In [24]:
res = ind.run(
    btc_price,
    window = 21)

In [25]:
res.value

Datetime
2023-09-30 05:16:00+00:00          NaN
2023-09-30 05:17:00+00:00          NaN
2023-09-30 05:18:00+00:00          NaN
2023-09-30 05:19:00+00:00          NaN
2023-09-30 05:20:00+00:00          NaN
                               ...    
2023-10-02 05:08:00+00:00    44.328440
2023-10-02 05:09:00+00:00    50.846761
2023-10-02 05:10:00+00:00    52.700563
2023-10-02 05:11:00+00:00    54.149025
2023-10-02 05:12:00+00:00    46.876559
Name: (21, Close), Length: 2869, dtype: float64

In [42]:
def custom_indicator(close, rsi_window = 14, ma_window= 50):
    rsi = vbt.RSI.run(close, window = rsi_window).rsi
    ma = vbt.MA.run(close, ma_window).ma
    return rsi

In [43]:
ind = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window','ma_window'],
    output_names=['value']
    ).from_apply_func(
            custom_indicator,
            rsi_window = 14,
            ma_window = 50
            )

In [44]:
res = ind.run(
    btc_price,
    rsi_window = 21,
    ma_window = 50)

In [47]:
print(res.value)

Datetime
2023-09-30 05:16:00+00:00          NaN
2023-09-30 05:17:00+00:00          NaN
2023-09-30 05:18:00+00:00          NaN
2023-09-30 05:19:00+00:00          NaN
2023-09-30 05:20:00+00:00          NaN
                               ...    
2023-10-02 05:08:00+00:00    44.328440
2023-10-02 05:09:00+00:00    50.846761
2023-10-02 05:10:00+00:00    52.700563
2023-10-02 05:11:00+00:00    54.149025
2023-10-02 05:12:00+00:00    46.876559
Name: (21, 50, Close), Length: 2869, dtype: float64


In [54]:
def custom_indicator(close, rsi_window = 14, ma_window= 50):
    rsi = vbt.RSI.run(close, window = rsi_window).rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    trend = np.where(rsi > 70, -1, 0)
    trend = np.where(rsi < 30, 1, trend)
    return trend

In [55]:
ind = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window','ma_window'],
    output_names=['value']
    ).from_apply_func(
            custom_indicator,
            rsi_window = 14,
            ma_window = 50
            )

In [56]:
res = ind.run(
    btc_price,
    rsi_window = 21,
    ma_window = 50)

In [64]:
print(res.value.unique())

[ 0 -1  1]


In [65]:
def custom_indicator(close, rsi_window = 14, ma_window= 50):
    rsi = vbt.RSI.run(close, window = rsi_window).rsi.to_numpy()
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    trend = np.where(rsi > 70, -1, 0)
    trend = np.where((rsi < 30) & (close < ma), 1, trend)
    return trend

In [66]:
ind = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window','ma_window'],
    output_names=['value']
    ).from_apply_func(
            custom_indicator,
            rsi_window = 14,
            ma_window = 50
            )

In [67]:
res = ind.run(
    btc_price,
    rsi_window = 21,
    ma_window = 50)

print(res.value.unique())

[ 0 -1  1]


In [68]:
entries = res.value == 1.0
exits = res.value == -1.0

In [69]:
pf = vbt.Portfolio.from_signals(btc_price,entries,exits)

In [71]:
pf.stats()

C:\Users\HP\anaconda3\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
C:\Users\HP\anaconda3\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
C:\Users\HP\anaconda3\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
C:\Users\HP\anaconda3\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2023-09-30 05:16:00+00:00
End                           2023-10-02 05:12:00+00:00
Period                                             2869
Start Value                                       100.0
End Value                                    100.294937
Total Return [%]                               0.294937
Benchmark Return [%]                           3.236058
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               0.998873
Max Drawdown Duration                            1084.0
Total Trades                                         24
Total Closed Trades                                  23
Total Open Trades                                     1
Open Trade PnL                                -0.068254
Win Rate [%]                                  65.217391
Best Trade [%]                                 0.192095
Worst Trade [%]                               -0

In [78]:
def custom_indicator(close, rsi_window = 14, ma_window= 50):
    close_5m = close.resample('5T').last()
    rsi = vbt.RSI.run(close_5m, window = rsi_window).rsi
    
    print(rsi)
    print(close)
    print(close_5m)
    return 1
    '''
    ma = vbt.MA.run(close, ma_window).ma.to_numpy()
    trend = np.where(rsi > 70, -1, 0)
    trend = np.where((rsi < 30) & (close < ma), 1, trend)
    return trend
    '''

In [79]:
ind = vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window','ma_window'],
    output_names=['value']
    ).from_apply_func(
            custom_indicator,
            rsi_window = 14,
            ma_window = 50,
            keep_pd = True
            )

In [80]:
res = ind.run(
    btc_price,
    rsi_window = 21,
    ma_window = 50)

rsi_window                        21
                               Close
Datetime                            
2023-09-30 05:15:00+00:00        NaN
2023-09-30 05:20:00+00:00        NaN
2023-09-30 05:25:00+00:00        NaN
2023-09-30 05:30:00+00:00        NaN
2023-09-30 05:35:00+00:00        NaN
...                              ...
2023-10-02 04:50:00+00:00  50.138829
2023-10-02 04:55:00+00:00  47.101839
2023-10-02 05:00:00+00:00  43.052370
2023-10-02 05:05:00+00:00  44.325419
2023-10-02 05:10:00+00:00  40.343079

[576 rows x 1 columns]
                                 Close
Datetime                              
2023-09-30 05:16:00+00:00  1671.835327
2023-09-30 05:17:00+00:00  1672.096680
2023-09-30 05:18:00+00:00  1672.276123
2023-09-30 05:19:00+00:00  1672.083618
2023-09-30 05:20:00+00:00  1672.021362
...                                ...
2023-10-02 05:08:00+00:00  1725.666748
2023-10-02 05:09:00+00:00  1725.615112
2023-10-02 05:10:00+00:00  1725.825684
2023-10-02 05:11:00+00:00  17

AssertionError: (1, 1) and (2869, 1) do not match

In [81]:
res.value

Datetime
2023-09-30 05:16:00+00:00    0
2023-09-30 05:17:00+00:00    0
2023-09-30 05:18:00+00:00    0
2023-09-30 05:19:00+00:00    0
2023-09-30 05:20:00+00:00    0
                            ..
2023-10-02 05:08:00+00:00    0
2023-10-02 05:09:00+00:00    0
2023-10-02 05:10:00+00:00    0
2023-10-02 05:11:00+00:00    0
2023-10-02 05:12:00+00:00    0
Name: (21, 50, Close), Length: 2869, dtype: int32